In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import re
import time
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf

In [128]:
df2 = pd.read_excel('data_final_consolidada_movies.xlsx')
df=df2[df2['mean_sentimiento']>0.3] 
df.head()

,name,promedio_ganancia_x,calificacion1_dec_x,calificacion2_dec_x,cluster,pp,Nombre_closter,budget,company,country,director,genre,gross,rating,day_released,month_released,year_released,released,runtime,score,star,votes,writer,year,hashtag,name_roten,calificacion1,calificacion2,synopsis,ruta,promedio_ganancia_y,calificacion1_dec_y,calificacion2_dec_y,max_sentimiento,min_sentimiento,mean_sentimiento,sentimiento,prueba
4,Now You See Me,1,3.0,4.0,2,"['charismatic', 'magician', 'atlas', 'jesse', ...",Cluster_3,75000000,Summit Entertainment,USA,Louis Leterrier,Crime,117723989,PG-13,31,5,2013,2013-05-31,115,7.3,Jesse Eisenberg,505432,Ed Solomon,2013,#NowYouSeeMe,now_you_see_me,51.0,70.0,charismatic magician atlas jesse eisenberg l...,https://www.rottentomatoes.com/m/now_you_see_me,1,3.0,4.0,0.9455,-0.7345,0.359890,Neutro,['see']
8,Blue Is the Warmest Color,0,4.0,4.0,0,"['french', 'teen', 'ad', 'le', 'exarchopoulos'...",Cluster_1,0,Quat'sous Films,France,Abdellatif Kechiche,Drama,2199675,NC-17,14,2,2014,2014-02-14,180,7.8,Léa Seydoux,107119,Abdellatif Kechiche,2013,#BlueIstheWarmestColor,blue_is_the_warmest_color,89.0,85.0,a french teen ad le exarchopoulos forms a de...,https://www.rottentomatoes.com/m/blue_is_the_w...,0,4.0,4.0,0.9291,-0.6551,0.397327,Neutro,"['blue', 'warmest', 'color']"
32,Beautiful Creatures,3,2.0,2.0,0,"['abused', 'boyfriend', 'tony', 'iain', 'glen'...",Cluster_1,60000000,Alcon Entertainment,USA,Richard LaGravenese,Drama,19452138,PG-13,14,2,2013,2013-02-14,124,6.2,Alice Englert,73570,Richard LaGravenese,2013,#BeautifulCreatures,beautiful_creatures,38.0,32.0,after being abused by her boyfriend tony iai...,https://www.rottentomatoes.com/m/beautiful_cre...,3,2.0,2.0,0.9701,-0.1860,0.647653,Positivo,"['beautiful', 'creatures']"
36,Elysium,1,NaN,NaN,2,"['sin', 'datos']",Cluster_3,115000000,TriStar Pictures,USA,Neill Blomkamp,Action,93050117,R,9,8,2013,2013-08-9,109,6.6,Matt Damon,366674,Neill Blomkamp,2013,#Elysium,elysium,NaN,NaN,Sin Datos,https://www.rottentomatoes.com/m/elysium,1,NaN,NaN,0.9726,-0.7959,0.302127,Neutro,['elysium']
39,The Family,1,1.0,2.0,0,"['ratting', 'mafia', 'cohorts', 'giovanni', 'm...",Cluster_1,30000000,EuropaCorp,USA,Luc Besson,Comedy,36918811,R,13,9,2013,2013-09-13,111,6.3,Robert De Niro,95229,Luc Besson,2013,#TheFamily,the_family,28.0,41.0,after ratting out his mafia cohorts giovanni ...,https://www.rottentomatoes.com/m/the_family_2013,1,1.0,2.0,0.9875,-0.9022,0.345277,Neutro,['family']


In [129]:
text = df.synopsis.str.cat(sep=" ")

In [130]:
vocab = sorted(set(text))
print(f'{len(vocab):,.0f} caracteres únicos')

39 caracteres únicos


In [131]:
text

'charismatic magician atlas  jesse eisenberg  leads a team of talented illusionists called the four horsemen  atlas and his comrades mesmerize audiences with a pair of amazing magic shows that drain the bank accounts of the corrupt and funnel the money to audience members  a federal agent  mark ruffalo  and an interpol detective  m lanie laurent  intend to rein in the horsemen before their next caper  and they turn to thaddeus  morgan freeman  a famous debunker  for help a french teen  ad le exarchopoulos  forms a deep emotional and sexual connection with an older art student  l a seydoux  she met in a lesbian bar after being abused by her boyfriend  tony  iain glen  dorothy  susan lynch  leaves him  at a bus station  she sees petula  rachel weisz  being attacked by her boyfriend  brian  tom mannion  and knocks him out  the girls drag him to petula s apartment  where he dies  panicked  the pair fakes a kidnapping to get a ransom from brian s psychotic brother  ronnie  maurice ro ves  w

In [132]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [134]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 512

In [135]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [136]:
text_as_int = np.array([char2idx[c] for c in text])

In [137]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1) # división entera

In [138]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [139]:
dataset = sequences.map(split_input_target)

In [140]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [141]:
BATCH_SIZE = 256

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [142]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (256, None, 64)           2496      
_________________________________________________________________
gru_14 (GRU)                 (256, None, 512)          887808    
_________________________________________________________________
gru_15 (GRU)                 (256, None, 512)          1575936   
_________________________________________________________________
dense_7 (Dense)              (256, None, 39)           20007     
Total params: 2,486,247
Trainable params: 2,486,247
Non-trainable params: 0
_________________________________________________________________


In [143]:
dataset.take(1)

<TakeDataset shapes: ((256, 100), (256, 100)), types: (tf.int64, tf.int64)>

In [144]:
for input_example_batch, target_example_batch in dataset.take(1):
    print("entrando")
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

entrando
(256, 100, 39) # (batch_size, sequence_length, vocab_size)


In [87]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (256, None, 64)           2496      
_________________________________________________________________
gru_8 (GRU)                  (256, None, 512)          887808    
_________________________________________________________________
gru_9 (GRU)                  (256, None, 512)          1575936   
_________________________________________________________________
dense_4 (Dense)              (256, None, 39)           20007     
Total params: 2,486,247
Trainable params: 2,486,247
Non-trainable params: 0
_________________________________________________________________


In [145]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [147]:
sampled_indices

array([36, 29,  8, 27,  5, 38, 12,  0, 37, 38,  6, 27, 19, 15,  6, 21, 17,
       33, 11, 36,  0, 22, 22,  3,  9, 26, 38, 30, 29,  3, 27, 33, 25,  7,
       36, 12, 34, 11, 13, 17, 29, 38, 29, 17,  4, 18, 10,  6, 15,  4, 15,
       36, 19,  0,  1,  4,  2, 22, 31, 25, 25, 30, 23,  8, 29, 33, 20, 37,
        2, 12,  1, 20,  4, 30, 36, 15, 36, 28, 38, 24, 36, 19, 15, 21,  6,
       18, 16, 26, 11,  8, 26,  3,  4, 38, 11, 29, 10, 31, 38, 22])

In [149]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'nds his life on the line after being pulled into the dangerous world of the escobar family business '

Next Char Predictions: 
 'xq7o4zS yz5ogc5ieuDx jj28nzrq2oum6xSvDaeqzqe3f95c3cxg 031jsmmrk7quhy1S0h3rxcxpzlxgci5fdnD7n23zDq9szj'


In [150]:
# Función de perdida para optimizar el modelo
def loss(labels, logits):
      return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [151]:
model.compile(optimizer='adam', loss=loss)


In [159]:
# Directory where the checkpoints will be saved
checkpoint_dir = 'training_checkpoints/'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

In [167]:
# Una epoca es la cantidad de pasos completos en el conjunto de datos de entrenamiento
EPOCHS=200
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)) # Comentar esta linea la primera vez 
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
1/1 [==============================] - 0s 82ms/step - loss: 0.3732
Epoch 2/200
1/1 [==============================] - 0s 68ms/step - loss: 0.3724
Epoch 3/200
1/1 [==============================] - 0s 71ms/step - loss: 0.3688
Epoch 4/200
1/1 [==============================] - 0s 70ms/step - loss: 0.3606
Epoch 5/200
1/1 [==============================] - 0s 73ms/step - loss: 0.3615
Epoch 6/200
1/1 [==============================] - 0s 80ms/step - loss: 0.3560
Epoch 7/200
1/1 [==============================] - 0s 67ms/step - loss: 0.3638
Epoch 8/200
1/1 [==============================] - 0s 68ms/step - loss: 0.3487
Epoch 9/200
1/1 [==============================] - 0s 74ms/step - loss: 0.3524
Epoch 10/200
1/1 [==============================] - 0s 61ms/step - loss: 0.3433
Epoch 11/200
1/1 [==============================] - 0s 66ms/step - loss: 0.3443
Epoch 12/200
1/1 [==============================] - 0s 63ms/step - loss: 0.3398
Epoch 13/200
1/1 [==============================]

In [168]:
p_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
p_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
p_model.build(tf.TensorShape([1, None]))

In [170]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return(start_string + ''.join(text_generated))

In [171]:
print(generate_text(p_model, start_string=u"fantasy"))

fantasy manci  radaie  mast rally their cosconer  drends that he couldn t possibly know a phar eve celebrating takes him under her wing matt scudd and march soon learn the hard wayher  his life os the local rufused out of wedly  good arl  an unable to digest food  despite the dire diagnosin  sam rockwell  at a time when he deads 15 venter  livistine siding  when roy and his new partner  1d horizon as leon rom  chisolm rescue   may  a john s cer poise are south  yet it was now en un inbleiather haupson  has rjends to recaise attical prowess drives the teens to succerd finkel  a struge comes through a smoky hearthor herman melvill the too  julia love  a defiant mortal named bee  and kazo  when roy and his new students find that they have one vangious wolln the survival of mankind hangs in the 1920s  raush towers  how erupal battle with zode as a hero and spendful che soon strockily  that there may nomes s comear  college shaten  so the yap br a fants as joche houre to ghat with his wife 

In [172]:
print(generate_text(p_model, start_string=u"horror"))

horrord marrixh  and their relationship is going  norman and carol try to res ut f man of her dead brother oner  until and copel ser  robert her hatpan  his seemingly clear path to success gets complicated when he  naw gravely ill  thus  aidan attempts to home school his offspring  as his chwell new  mifha cath compunity  especially thatul her pore the survival of mankind hangs in the balance when set  bet american heiress and a revolutionary want it op over to receive  her longing a job with his us coseigher wather finker pasker  josh juman  a chain of events unfold gredfie devote their existence to ralis es extwent family ties  incredible work eproc story of a conippant for a prostitht compite  emme action comedican of finder write  mobey wilit fall of their home  the couple to the congo  rom plans to capture tarzan and deliver him to an old enemy in exchnoweethem to gith she s been groomed for stardom all her life by an overbearing mothe magh  requeying a younger woman  her friend j

In [173]:
print(generate_text(p_model, start_string=u"hero"))

herousm as he joins a huge army of elite warriors to catch the kniversent s time  with her go act setulaging bark  she gets a young a sexy blond dancer  juno temple  she invition to the fright heaves a holm warrior  matt damon  is a wither takes three men on a lifeboat towte  billakn in ender the grading unis of rending actor  lives in tho documend his hem surard of hin dromer to a predoces for ghinger  wa5nerf reunie  harger  julo  s their ore and ollfe  their old relen  but amwer contin  wright  a 17 year old student  loses her virginity zuch a high school sweeter for activities evertheid movement and co somelaghatif  thaughter  gugu mbatha raw  of a british admiral plays an important role in the campaign to abolish sluses and other farmers  knig tims en  that there may not be a treic ot warld in cept a both the will wry to los  jass deadan getara  kaight reaves her unfole be with his molenta cullace  when heran agant  marh lauren  erbeautiful  with the emergence of electroo love in 

In [174]:
print(generate_text(p_model, start_string=u"super"))

super she learns that her classmates secretly  arexpigate  he forces the women to cut him in on their strips that the crime sparls a jives with a raing the 1972 world chess championship a young american in paris works an un  nalpy fart them tee and poss spett  pake atain  as sod he weekend to clean out the junk friends in the case unexpectedly  afternct his chief technical officer  jason bateman  must rally their co worker from her as a lifeboa makic rovicu  her boyfriend  jro win a national title  all spoute realizes the boys pore interave the woman he lives  the francory  jessiey  as lives will be thrown into a terrifying tailspin  at first  simon doesn t recognize gor a her aloug  mark daugh  who show dis daks and nivily a stripp e ply on the 1960s  eddie mannix is busy at workon storys his life on the line after being pulled into the dangerous world of the escovery boyer  but he has a problem  college friends lance  morris chestnut  harpy spend  a wichard linking parizan batte s pa

In [175]:
print(generate_text(p_model, start_string=u"magic"))

magic workment  where he began in stand up comedy  andre allen  chris rock  hit the big time as the state of 1924s house  a man  logan marshall grotection s spaces the cound two cruan world chess a just called and ong them adrifting a housed residents of the corrupt and funnel the money to audience members  a federal agent  mark ruffalo  matt mamaud of hem freak who doesn t tell her parental disapprovale  dreach begina travel writer  jah  causin her to the friends dian wilas  the government sunder  ant relocate to uall  he  namee a prostitht have come and gone  which contrition  as chrest battle yet is about to boye in her boyfriend  trent  steven one cares truar  woody allen  hires out his marriage to sunaina draws near  sonnghal and relocate to a french town  despite the bestorts  the hudans  peough when he has an affair with a trobbly women who shakent acle  bat mogel priest accused out of wedlock and wasker hoit papingint a for dark agrion wook will save their jobs Sin Datos on feb

In [176]:
print(generate_text(p_model, start_string=u"travel")) 

travely takes the charoughter  gugut in cear  with the emergence of electroo  john d leo  can t help but resort to doing things the  family and ldave the world and rescuey his jog when he mistover  his lifes parter  at a bus station  face  when losh out to save her senior year from becoming ar aca the wonde when he hes and puts after a man  lich strenga  kristing unexpected personal responses  he begins to understand her pare  mandyy patink a herdy way arroged sharres  factor  locess ding ane kecomes the couple to the congo  rom plans to capture tarzan and deliver him to an old enemy in exchange duss into the winawer  liam neeson  formerly part of the noum  a firring murder  with story  however  madal  julianne howard  robyn  sanaa lathan  jordan  nin pasonts t foo hot him  as the brefore  nthel heart surper  forced to flen he inner cipy of a soviet for the takes a loc  and the chief suspect  they have in enriar with  they wind up knocking supermones  shatress inner humanity and the re